# Monterey Bay Curvilinear Grid README #

GCCOM Monterey Bay Grid Generation Repository

![Monterey Bay Basemap](export/mont-basemap.png)

## Repository Details ##

* Download/Format bathymetry data
* Create KML in Google Earth
* Parse/Extract coordinates from KML
* Activate python environment
* Create curvilinear grid using GRIDGEN Library
* Visualize output


### Download/Format bathymetry data ###

Download gridded bathymetry from a variety of sources. Typically data are formatted as ASCII or ESRI Grid format.

* [NOAA Coastal Elevation Models] (https://www.ngdc.noaa.gov/mgg/coastal/coastal.html)
* [SCCOOS Bathymetry] (http://www.sccoos.org/data/bathy/?region=monterey)

For this example, download and unzip ASCII file from SCCOOS


```
$ cd ./input
$ wget http://www.sccoos.org/data/bathy/data/monterey.zip
$ unzip ./monterey.zip
```

This will create the comma-delimited lon,lat,z file: monterey.xyz.
We need the file to be in UTM coordinates, so project from GEODETIC (lon/lat) to UTM Zone 10 meters. This is accomplished with python script ../scripts/bathy-project.py, which will output the bathy file: mont_bathy.xyz


In [ ]:
```
$ cd ./input
$ python ../scripts/bathy-project.py ./monterey.xyz
$ cd ..

  Reading Geo File:../input/monterey.xyz
  Projecting Data to UTM Coordinates
  Created UTM File:../input/mont_bathy.xyz
```

Below is actual python code from bathy-project.py

In [11]:
#- Initialize libs for this notebook
import os, sys, shutil
import math
import numpy as np
import netCDF4, datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
sys.path.append("./includes")
sys.path.append("./scripts")
from gccom_utils import *
from gccom_plot import *
from gis_utils import *
from ll2utm import *
%run ./includes/gccom_utils.py
%run ./includes/gis_utils.py
%run ./includes/gccom_plot.py
#%run ./scripts/ll2utm.py

In [12]:
#- Project data using bathy-project.py
input_file = './input/monterey.xyz'
#- Open up file and read coordinates
print("Reading Geo File:"+input_file)
fid = open(input_file,'r')
lon,lat,z = [],[],[]
for line in fid:
    line = line.strip()
    cols = line.split(',')
    lon.append(float(cols[0]))
    lat.append(float(cols[1]))
    z.append(float(cols[2]))
fid.close()


Reading Geo File:./input/monterey.xyz


In [13]:
print("Projecting Data to UTM Coordinates")
zone, easting, northing = LL_to_UTM(23, lat, lon)

#- Write output
out_file = "./input/mont_bathy.xyz"
ofid = open(out_file,'w')
for i in range(len(easting)):
    ofid.write('%9.2f %10.2f %8.2f\n' % (easting[i],northing[i],z[i]))
ofid.close()
print("Created UTM File:"+out_file)


Projecting Data to UTM Coordinates
Created UTM File:./input/mont_bathy.xyz


In [20]:
print("Input (lat,long): {}, {}\nOutput (x,y): {} {}".format(
            lat[0], lon[0], northing[0], easting[0]))


Input (lat/long): 37.25, -122.75
Output (x/y): 4122635.547197988 522170.99832325714


### Digitize Bounding Polygon in Google Earth ###

* ./input/mont-outline.kml: KML file digitized from Google Earth.
* Screengrabs of process: ./export/screengrabs/

![Google Earth KML Creation](export/screengrabs/ge_mont_2.png)


### Parse KML File ###

* Run python script to format/parse digitized kml polygon:

```
$ python ./scripts/parse_kml.py ./input/mont-outline.kml
```

* INPUT: ./input/mont-outline.kml
* OUTPUT: ./input/mont-outline_geo.xy

Below are the python commands from parse_kml.py:

In [23]:
#--------------------------------------------------------
#- File: parse_kml.py
#- Description: Extract coordinates from Google Earth KML
#-              and convert to UTM, saving results.
#- Author: Randy Bucciarelli
#- Date: June 15, 2017
#--------------------------------------------------------
import sys
from xml.dom.minidom import parseString

#- Define source files
src_dir = './input/'
kml_file = src_dir+'mont-outline.kml'

#- Open kml file and read into memory
file = open(kml_file)
theData = file.read()
file.close()

#-Load data string into DOM
theDom = parseString(theData)

lats,lons,verts = [],[],[]
#- Loop through dom and find coordinates
for d in theDom.getElementsByTagName('coordinates'):
    #- Get all the vertices in first polygon
    positions = d.firstChild.data.split()
    for p in positions:
        coords = p.split(',') 
        lats.append(float(coords[1]))
        lons.append(float(coords[0]))
        verts.append(0)

#- Write geographic coords to outfile
out_file = kml_file[0:len(kml_file)-4]+'_geo.xy'
thefile = open(out_file, 'w')
for i in range(len(lats)):
    thefile.write("%13.8f\t%13.8f\t%d\n" % (lons[i],lats[i],verts[i]))
thefile.close()
print("Created "+out_file)


Created ./input/mont-outline_geo.xy


### Create Curvilinear Grid ###

* Activate correct python environment

```
$ source activate grid
```

* Run python script ./scripts/create_grid.py to Create curvilinear grid from geographic coords

```
(grid) $ python ./scripts/create_grid.py ./input/mont-outline_geo.xy

Created plot: ./export/mont-grid.png
Created grid corner nodes: ./output/mont_grid.xy
Created grid center nodes: ./output/mont_grid_CE.xy

```

* INPUT: ./input/mont-outline_geo.xy, ./input/mont-vertices.txt
* OUTPUT: ./output/mont-outline_utm.xy, ./output/mont_grid.xy, ./output/mont_grid_CE.xy, ./export/mont-grid.png

![Output Curvilinear Grid](export/mont-grid.png)


### Interpolate Bathymetry Onto Curvilinear Grid ###
* Execute BASH shell script (./scripts/gridgen_gridbathy.sh) 
* * Interpolates source bathymetry onto curvlinear grid center points
* * Requires GRIDBATHY code (./software/gridutils/gridbathy)
* INPUT: ./input/mont_bathy.dat, ./output/mont_grid.xy
* OUTPUT: ./output/output_bathy_CE, ./output/output_grid_CE

```
$ cd ./scripts
$ ./gridgen_gridbathy.sh
$ cd ..
```

![Interpolated Bathymetry 2D](export/mont-bathy.png)


### Format 2-D curvilinear grid for GCCOM ###
* Python script (./scripts/gridgen2gcom2d.py) 
* * Merges grid center points with bathymetry
* * Formats for use in GCCOM
* * Creates Problem Size file (./output/ProbSize.dat) with Grid Dimensions
* INPUT: ./output/output_bathy_CE, ./output/output_grid_CE
* OUTPUT: ./output/gcom_grid2d.dat, ./output/ProbSize_mont.dat

```
$ python ./scripts/gridgen2gcom2d.py

NX = 49; NY = 24
Created 2-D Grid File: ./output/gcom_grid2d.dat
Created GCCOM ProbSize File: ./output/ProbSize_mont.dat

```


### Generate 3-D mesh from 2-D ###
* Python script (./scripts/gen_gcom_3dgrid.py) 
* * Python script to generate 3d mesh from 2d
* * Read mesh dimensions from ProbSize.dat file
* INPUT: ./output/gcom_grid2d.dat, ./output/ProbSize_mont.dat
* OUTPUT: ./output/Grid_lj.dat

```
$ python ./scripts/gen_gcom_3dgrid.py

Mesh Dimensions: 49x24x13
```

![3-D Curvilinear Mesh](export/mont-bathy-3d.png)

### Visualize Output Grids With Bathymetry ###

To visualize interpolated 2D bathymetry:

```
$ python ./scripts/plot_bathy.py

```

To visualize 3D curvilinear mesh:

```
$ python ./scripts/plot_bathy_3d.py
```

### Visualize Y-Z (J-K) Plane ###
* Python script (./scripts/plot_face.py) 

![3-D Curvilinear Mesh Face](export/mont-face.png)
